# V3 Ultimate - 5-Model Ensemble Trading System

**Target Performance**: 75-78% win rate, 1.8+ Sharpe ratio

**Runtime**: ~49 hours on Colab Pro+ A100

---

## Setup Instructions

1. **Subscribe to Colab Pro+**: https://colab.research.google.com/signup
2. **Select A100 GPU**: `Runtime > Change runtime type > A100 GPU`
3. **Run cells in order**

---

## Step 0: Mount Google Drive & Setup

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

print('✅ Google Drive mounted')

In [ ]:
# Check GPU
!nvidia-smi

import torch
print(f"\nPyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f}GB")
    
    if 'A100' in torch.cuda.get_device_name(0):
        print('\n✅ A100 GPU detected - optimal performance!')
    else:
        print('\n⚠️  Not A100 - training may be slower')
else:
    print('\n❌ No GPU detected!')

In [ ]:
# Create directory structure
!mkdir -p /content/drive/MyDrive/crpbot/v3_ultimate
!mkdir -p /content/drive/MyDrive/crpbot/data/raw
!mkdir -p /content/drive/MyDrive/crpbot/data/features
!mkdir -p /content/drive/MyDrive/crpbot/models
!mkdir -p /content/drive/MyDrive/crpbot/backtest

print('✅ Directory structure created')

## Step 1: Install Dependencies

In [ ]:
# Install ML libraries
!pip install -q xgboost lightgbm catboost pytorch-tabnet
!pip install -q pandas numpy pyarrow scikit-learn
!pip install -q onnx onnxruntime skl2onnx
!pip install -q boto3 ccxt tqdm joblib shap

# Install TA-Lib
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install
%cd ..
!pip install -q ta-lib

print('\n✅ All dependencies installed')

## Step 2: Upload V3 Ultimate Scripts

**IMPORTANT**: Upload these files to `/content/drive/MyDrive/crpbot/v3_ultimate/`:

- `00_run_v3_ultimate.py`
- `01_fetch_data.py`
- `02_engineer_features.py`
- `03_train_ensemble.py`
- `04_backtest.py`
- `05_export_onnx.py`

You can upload via:
1. Google Drive web interface
2. Or use the file upload cell below

In [ ]:
# Upload files (optional - alternative to Drive upload)
from google.colab import files

print('Select all 6 Python files to upload')
uploaded = files.upload()

# Move to correct directory
for filename in uploaded.keys():
    !mv {filename} /content/drive/MyDrive/crpbot/v3_ultimate/

print('\n✅ Files uploaded')

In [ ]:
# Verify files exist
!ls -lh /content/drive/MyDrive/crpbot/v3_ultimate/*.py

## Step 3: Run V3 Ultimate Pipeline

**Choose one option:**

### Option A: Run All Steps (49 hours)

In [ ]:
# Navigate to scripts directory
%cd /content/drive/MyDrive/crpbot/v3_ultimate

# Run entire pipeline
!python 00_run_v3_ultimate.py

### Option B: Run Individual Steps

Run each step separately (recommended for monitoring progress)

In [ ]:
# Step 1: Data Collection (12 hours)
%cd /content/drive/MyDrive/crpbot/v3_ultimate
!python 01_fetch_data.py

In [ ]:
# Step 2: Feature Engineering (4 hours)
%cd /content/drive/MyDrive/crpbot/v3_ultimate
!python 02_engineer_features.py

In [ ]:
# Step 3: Train Ensemble (24 hours)
%cd /content/drive/MyDrive/crpbot/v3_ultimate
!python 03_train_ensemble.py

In [ ]:
# Step 4: Backtest (8 hours)
%cd /content/drive/MyDrive/crpbot/v3_ultimate
!python 04_backtest.py

In [ ]:
# Step 5: Export ONNX (1 hour)
%cd /content/drive/MyDrive/crpbot/v3_ultimate
!python 05_export_onnx.py

## Step 4: Monitor Progress

Use these cells to monitor training progress

In [ ]:
# Check checkpoint status
import json

checkpoint_path = '/content/drive/MyDrive/crpbot/v3_ultimate_checkpoint.json'

try:
    with open(checkpoint_path, 'r') as f:
        checkpoint = json.load(f)
    
    print('📊 V3 Ultimate Progress\n')
    print(f"Last Updated: {checkpoint['timestamp']}")
    print(f"Last Completed Step: {checkpoint['last_completed_step']}\n")
    
    print('Steps:')
    for step, success in checkpoint['results'].items():
        status = '✅' if success else '❌'
        print(f"  {status} Step {step}")
        
except FileNotFoundError:
    print('⚠️  No checkpoint found yet')

In [ ]:
# Monitor GPU usage (run in separate cell)
!watch -n 1 nvidia-smi

## Step 5: Validate Results

In [ ]:
# Check training metadata
import json

metadata_path = '/content/drive/MyDrive/crpbot/models/metadata.json'

with open(metadata_path, 'r') as f:
    metadata = json.load(f)

print('📊 Training Results\n')
print(f"Timestamp: {metadata['timestamp']}")
print(f"Features: {metadata['num_features']}")
print(f"Duration: {metadata['duration_seconds']/3600:.1f} hours\n")

print('Metrics:')
print(f"  Accuracy: {metadata['metrics']['accuracy']:.1%}")
print(f"  AUC: {metadata['metrics']['auc']:.3f}")
print(f"  ECE: {metadata['metrics']['ece']:.4f}")
print(f"  Win Rate: {metadata['metrics']['win_rate']:.1%}\n")

if metadata['gates_passed']:
    print('✅ ALL TRAINING GATES PASSED')
else:
    print('⚠️  Some training gates failed')

In [ ]:
# Check backtest results
import json

backtest_path = '/content/drive/MyDrive/crpbot/backtest/backtest_summary.json'

with open(backtest_path, 'r') as f:
    backtest = json.load(f)

print('📈 Backtest Results\n')
print(f"Duration: {backtest['duration_seconds']/3600:.1f} hours\n")

metrics = backtest['metrics']
print('Performance:')
print(f"  Total Trades: {metrics['total_trades']:,}")
print(f"  Win Rate: {metrics['win_rate']:.1%}")
print(f"  Sharpe Ratio: {metrics['sharpe_ratio']:.2f}")
print(f"  Max Drawdown: {metrics['max_drawdown']:.1%}")
print(f"  Total Return: {metrics['total_return']:.1%}")
print(f"  Profit Factor: {metrics['profit_factor']:.2f}\n")

if backtest['gates_passed']:
    print('✅ ALL BACKTEST GATES PASSED')
else:
    print('⚠️  Some backtest gates failed')

## Step 6: Download Models

In [ ]:
# Zip all models and results
!zip -r v3_ultimate_models.zip /content/drive/MyDrive/crpbot/models/
!zip -r v3_ultimate_backtest.zip /content/drive/MyDrive/crpbot/backtest/

print('✅ Files zipped')

# Download
from google.colab import files
files.download('v3_ultimate_models.zip')
files.download('v3_ultimate_backtest.zip')

print('✅ Download complete!')

## Step 7: View Performance Charts

In [ ]:
# Plot equity curve
import pandas as pd
import matplotlib.pyplot as plt

equity_path = '/content/drive/MyDrive/crpbot/backtest/equity_curve.csv'
equity = pd.read_csv(equity_path)

plt.figure(figsize=(14, 6))
plt.plot(equity['index'], equity['equity'])
plt.title('V3 Ultimate - Equity Curve (5 Years Backtest)', fontsize=16)
plt.xlabel('Candle Index')
plt.ylabel('Capital ($)')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Initial Capital: ${equity['equity'].iloc[0]:,.2f}")
print(f"Final Capital: ${equity['equity'].iloc[-1]:,.2f}")
print(f"Total Return: {(equity['equity'].iloc[-1] / equity['equity'].iloc[0] - 1):.1%}")

In [ ]:
# Plot trade distribution
import pandas as pd
import matplotlib.pyplot as plt

trades_path = '/content/drive/MyDrive/crpbot/backtest/backtest_results.csv'
trades = pd.read_csv(trades_path)

# Filter closed trades
closed_trades = trades.dropna(subset=['pnl'])

plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.hist(closed_trades['pnl_pct'] * 100, bins=50, edgecolor='black')
plt.title('Trade P&L Distribution')
plt.xlabel('P&L (%)')
plt.ylabel('Count')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.hist(closed_trades['confidence'], bins=50, edgecolor='black')
plt.title('Trade Confidence Distribution')
plt.xlabel('Confidence')
plt.ylabel('Count')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nTrade Statistics:")
print(f"  Total Trades: {len(closed_trades):,}")
print(f"  Winning: {len(closed_trades[closed_trades['pnl'] > 0]):,}")
print(f"  Losing: {len(closed_trades[closed_trades['pnl'] <= 0]):,}")
print(f"  Avg Confidence: {closed_trades['confidence'].mean():.1%}")

---

## 🎉 V3 Ultimate Complete!

**Next Steps:**
1. Download models and backtest results
2. Verify validation gates passed
3. Deploy to production infrastructure
4. Configure real-time data feeds
5. Start paper trading

---